# Fine-tuning de BERT -> NER
Ce programme a pour but d'entrainer les dernières couches d'un modele pré-entrainé avec des données provenant d'un dataset transformé.    
Ainsi, il pourra effectuer une tâche de classification qui consiste à détecter les entités de départ et d'arrivée dans une phrase (un ordre de réservation de voyages) en français.

### Observer les données

In [1]:
import pandas as pd
import numpy as np

# Define the paths to the CSV files
file_path_train = '/home/jovyan/data/reservation-first-dataset-train.csv'
file_path_test = '/home/jovyan/data/reservation-first-dataset-test.csv'

# Load the data
try:
    data_train = pd.read_csv(file_path_train).fillna('')  # Handle possible NaN values
    data_test = pd.read_csv(file_path_test).fillna('')    # Handle possible NaN values
    print("-> loaded successfully")
except FileNotFoundError:
    print("Error : please verify the file paths")

print(f"Train sentences: {len(data_train)}")
print(f"Test sentences: {len(data_test)}")
print(data_train.columns)
print(data_test.columns)

print("-> Head train data")
print(data_train.head())
print("-> Head test data")
print(data_test.head())

-> loaded successfully
Train sentences: 207
Test sentences: 20
Index(['Phrase', 'Départ', 'Arrivée'], dtype='object')
Index(['Phrase', 'Départ', 'Arrivée'], dtype='object')
-> Head train data
                                              Phrase                 Départ  \
0  montre-moi les trains dimanche allant de Jarvi...  Jarville-la-Malgrange   
1  quels trains voyagent d'Alençon à Corbeil-Esso...                Alençon   
2  montre-moi les trains pour Saint-Avold depuis ...               Xertigny   
3  montrer les trains de Gargan à Valdahon Camp M...                 Gargan   
4  quels trains sont disponibles de Montbard à Sa...               Montbard   

                   Arrivée  
0      La Bassée-Violaines  
1         Corbeil-Essonnes  
2              Saint-Avold  
3  Valdahon Camp Militaire  
4      Saint-Romain-le-Puy  
-> Head test data
                                              Phrase               Départ  \
0  S'il vous plaît, donnez-moi des trains d'Imphy...            

## Tokenizer et encoder - First essaie

In [11]:
import os
import pandas as pd
from transformers import BertTokenizer, BertTokenizerFast, TFBertForTokenClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # Masquer les avertissements
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 0 = toutes les logs, 1 = info logs masqués, 2 = info et warning masqués, 3 = tout masqué

file_path_train = '/home/jovyan/data/reservation-first-dataset-train.csv'
file_path_test = '/home/jovyan/data/reservation-first-dataset-test.csv'
data_train = pd.read_csv(file_path_train).fillna('')
data_test = pd.read_csv(file_path_test).fillna('')

# Encodage des phrases et des labels correspondants
def encode_data(data, tokenizer, label_encoder, max_length=36): # data = CSV en input / max_length = Longueur maximale des phrases
    tokens = []
    labels = []

    for i, row in data.iterrows():
        phrase = row['Phrase']
        dep = row['Départ']
        arr = row['Arrivée']

        if i < 4:  # print 4 lignes
            print(f"\n- Ligne {i+1}")
            print("Phrase originale :", phrase)

        tokenized_input = tokenizer.encode_plus(
            phrase,
            add_special_tokens=True,  # [CLS] au début et [SEP] à la fin
            return_offsets_mapping=True,
            return_tensors="tf",  # retourne les tokens en tensors
            max_length=max_length,
            truncation=True,  # si la phrase est +longue que max_length
            padding="max_length"  # padding pour que ttes les séquences aient la même longueur
        )

        tokenized_text = tokenizer.convert_ids_to_tokens(tokenized_input.input_ids[0])
        offsets = tokenized_input['offset_mapping'].numpy()[0]
        label_list = ['O'] * len(tokenized_text)

        if i < 4:
            print("Tokens encodés :", tokenized_text)
            # print("Offsets :", offsets)

        for j, (start, end) in enumerate(offsets):
            if start and end and start != end:
                token_str = phrase[start:end]
                if token_str in dep:
                    label_list[j] = 'B-DEP'
                elif token_str in arr:
                    label_list[j] = 'B-ARR'

        if i < 4:
            print("Labels après encodage des entités :", label_list)

        # Conversion des labels en ids
        label_ids = label_encoder.transform(label_list)
        tokens.append(tokenized_input.input_ids.numpy()[0])
        labels.append(label_ids)

        if i < 4:
            print("IDs des label", label_ids)

    print("\n// Encoding completed")
    return np.array(tokens), np.array(labels)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased') # Version fast du tokenizer BERT

print("\n// Preparation of labels ['O', 'B-DEP', 'B-ARR']")
unique_labels = ['O', 'B-DEP', 'B-ARR']
label_encoder = LabelEncoder()
label_encoder.fit(unique_labels)
print("-> mapping labels et ids :", {label: idx for idx, label in enumerate(label_encoder.classes_)})

print("\n// Starting train data encoding...")
train_tokens, train_labels = encode_data(data_train, tokenizer, label_encoder, max_length=36)
print("\n// Starting test data encoding...")
test_tokens, test_labels = encode_data(data_test, tokenizer, label_encoder, max_length=36)

print("\n// Shape")
print("- train tokens", train_tokens.shape)
print("- train labels", train_labels.shape)
print("- test tokens", test_tokens.shape)
print("- test labels", test_labels.shape)




// Preparation of labels ['O', 'B-DEP', 'B-ARR']
-> mapping labels et ids : {'B-ARR': 0, 'B-DEP': 1, 'O': 2}

// Starting train data encoding...

- Ligne 1
Phrase originale : montre-moi les trains dimanche allant de Jarville-la-Malgrange à La Bassée-Violaines en première classe sans correspondance partant l'après midi
Tokens encodés : ['[CLS]', 'montre', '-', 'moi', 'les', 'trains', 'dimanche', 'allant', 'de', 'Jar', '##ville', '-', 'la', '-', 'Mal', '##gra', '##nge', 'à', 'La', 'Bass', '##ée', '-', 'Viola', '##ines', 'en', 'première', 'classe', 'sans', 'correspond', '##ance', 'part', '##ant', 'l', "'", 'après', '[SEP]']
Labels après encodage des entités : ['O', 'O', 'B-DEP', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DEP', 'B-DEP', 'B-DEP', 'B-DEP', 'B-DEP', 'B-DEP', 'B-DEP', 'B-DEP', 'O', 'B-ARR', 'B-ARR', 'B-ARR', 'B-DEP', 'B-ARR', 'B-ARR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DEP', 'O', 'O', 'O']
IDs des label [2 2 1 2 2 2 2 2 2 1 1 1 1 1 1 1 1 2 0 0 0 1 0 0 2 2 2 2 2 2 2 2 1 2 2 2]



#### Analyse First essaie

Ici on peut voir que les tokens et les labels sont désalignés à cause de la fonction encode_data.   
Elle vérifie si le token_str est contenu dans le nom de la ville de départ ou d'arrivée avec l'opérateur "in".   
Par ex que le tiret '-' peut être présent dans un nom de ville mais également à n'importe quel endroit du texte.   

Pour résoudre ce problème, la proposition est de trouver la position dans la phrase avec les ids.   
Et ainsi de vérifier pour chaque token si son offset chevauche l'une des entités. Si oui, le label approprié sera attribué.   

Enfin le schéma de labellisation 'BIO' (Begin, Inside, Outside) sera implémenter pour mieux gérer les multi-tokens.

In [13]:
import os
import pandas as pd
from transformers import BertTokenizer, BertTokenizerFast, TFBertForTokenClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) # Masquer les avertissements
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # 0 = toutes les logs, 1 = info logs masqués, 2 = info et warning masqués, 3 = tout masqué

file_path_train = '/home/jovyan/data/reservation-first-dataset-train.csv'
file_path_test = '/home/jovyan/data/reservation-first-dataset-test.csv'
data_train = pd.read_csv(file_path_train).fillna('')
data_test = pd.read_csv(file_path_test).fillna('')

print("\n// Preparation of labels ['O', 'B-DEP', 'I-DEP', 'B-ARR', 'I-ARR']")
unique_labels = ['O', 'B-DEP', 'I-DEP', 'B-ARR', 'I-ARR']
label_encoder = LabelEncoder()
label_encoder.fit(unique_labels)
print("-> mapping labels et ids :", {label: idx for idx, label in enumerate(label_encoder.classes_)})

# Encodage des phrases et des labels correspondants
def encode_data(data, tokenizer, label_encoder, max_length=36):  # data = CSV en input / max_length = Longueur maximale des phrases
    tokens = []
    labels = []
    
    print("Starting data encoding...")
    for i, row in data.iterrows():
        phrase = row['Phrase']
        dep = row['Départ']
        arr = row['Arrivée']

        if i < 4:  # print 4 lignes
            print(f"\n- Ligne {i+1}")
            print("Phrase originale :", phrase)

        dep_positions = []
        arr_positions = []

        start = 0  # Rechercher toutes les occurrences de départ
        while True:
            idx = phrase.find(dep, start)
            if idx == -1:
                break
            dep_positions.append((idx, idx + len(dep)))
            start = idx + len(dep)

        start = 0  # Rechercher toutes les occurrences d'arrivée
        while True:
            idx = phrase.find(arr, start)
            if idx == -1:
                break
            arr_positions.append((idx, idx + len(arr)))
            start = idx + len(arr)

        tokenized_input = tokenizer.encode_plus(
            phrase, add_special_tokens=True, return_offsets_mapping=True, max_length=max_length, truncation=True, padding="max_length", return_tensors="tf"
        )

        tokenized_text = tokenizer.convert_ids_to_tokens(tokenized_input.input_ids[0])
        offsets = tokenized_input['offset_mapping'].numpy()[0]
        label_list = ['O'] * len(tokenized_text)

        if i < 4:
            print("Tokens encodés :", tokenized_text)

        for j, (offset_start, offset_end) in enumerate(offsets): # Assignation des labels aux tokens
            if offset_start == 0 and offset_end == 0:
                continue  # Token de padding
            token_label = 'O'

            # Vérifier si le token chevauche une entité de départ
            for entity_start, entity_end in dep_positions:
                if (offset_start >= entity_start) and (offset_end <= entity_end):
                    if offset_start == entity_start:
                        token_label = 'B-DEP'
                    else:
                        token_label = 'I-DEP'
                    break

            # Vérifier si le token chevauche une entité d'arrivée
            for entity_start, entity_end in arr_positions:
                if (offset_start >= entity_start) and (offset_end <= entity_end):
                    if offset_start == entity_start:
                        token_label = 'B-ARR'
                    else:
                        token_label = 'I-ARR'
                    break

            label_list[j] = token_label

        if i < 4:
            print("Labels après encodage des entités :", label_list)

        label_ids = label_encoder.transform(label_list)
        tokens.append(tokenized_input.input_ids.numpy()[0])
        labels.append(label_ids)

        if i < 4:
            print("IDs des labels :", label_ids)

    print("\n// Encoding completed")
    return np.array(tokens), np.array(labels)


tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased') # Version fast du tokenizer BERT

print("\n// Starting train data encoding...")
train_tokens, train_labels = encode_data(data_train, tokenizer, label_encoder, max_length=36)
print("\n// Starting test data encoding...")
test_tokens, test_labels = encode_data(data_test, tokenizer, label_encoder, max_length=36)


# Chargement du modèle
model = TFBertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_encoder.classes_))
print("-> modèle chargé avec", len(label_encoder.classes_), "labels")

# Configuration de l'entraînement 
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Entraînement
model.fit(train_tokens, train_labels, epochs=3, batch_size=16, validation_split=0.1)

# Évaluation
model.evaluate(test_tokens, test_labels)



// Preparation of labels ['O', 'B-DEP', 'I-DEP', 'B-ARR', 'I-ARR']
-> mapping labels et ids : {'B-ARR': 0, 'B-DEP': 1, 'I-ARR': 2, 'I-DEP': 3, 'O': 4}

// Starting train data encoding...
Starting data encoding...

- Ligne 1
Phrase originale : montre-moi les trains dimanche allant de Jarville-la-Malgrange à La Bassée-Violaines en première classe sans correspondance partant l'après midi
Tokens encodés : ['[CLS]', 'montre', '-', 'moi', 'les', 'trains', 'dimanche', 'allant', 'de', 'Jar', '##ville', '-', 'la', '-', 'Mal', '##gra', '##nge', 'à', 'La', 'Bass', '##ée', '-', 'Viola', '##ines', 'en', 'première', 'classe', 'sans', 'correspond', '##ance', 'part', '##ant', 'l', "'", 'après', '[SEP]']
Labels après encodage des entités : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DEP', 'I-DEP', 'I-DEP', 'I-DEP', 'I-DEP', 'I-DEP', 'I-DEP', 'I-DEP', 'O', 'B-ARR', 'I-ARR', 'I-ARR', 'I-ARR', 'I-ARR', 'I-ARR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
IDs des labels : [4 4 4 4 4 4

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-> modèle chargé avec 5 labels
Epoch 1/3
12/12 [==============================] - 128s 8s/step - loss: 0.5995 - accuracy: 0.7591 - val_loss: 0.3437 - val_accuracy: 0.8823
Epoch 2/3
12/12 [==============================] - 48s 4s/step - loss: 0.2357 - accuracy: 0.9246 - val_loss: 0.0867 - val_accuracy: 0.9828
Epoch 3/3
1/1 [==============================] - 1s 1s/step - loss: 0.0997 - accuracy: 0.9583


[0.09974091500043869, 0.9583333134651184]

In [14]:
# Et là avec cette opti il n'y a pas d'erreurs dans la tokenization et l'accuracy est ultra bonne.

# Reste A Faire
# - format de sortie : 
#     return f"sentenceID, Departure: {' '.join(depart)}, Destination: {' '.join(arrivee)}"
#     return "sentenceID, Code=['UNKNOWN']"  # + la logique pour NOT_FRENCH et NOT_TRIP
# - predictions
# - metrics